# Association

## C1C2

In [1]:
%%writefile src/C1.sol
//SPDX-License-Identifier: GPL-3.0-or-later
pragma solidity ^0.8.0;
//pragma solidity ^0.6;
//pragma solidity 0.4.21;

contract C1 {
    uint128 v1;
    function set(uint128 _v1) public {
        v1=_v1;
    }
    function get() public view returns(uint128) {
        return v1;
    }
    function get7() public pure returns(uint128) {
        return 7;
    }
}

Writing src/C1.sol


In [2]:
!solc-windows.exe src/C1.sol --combined-json abi,bin > src/C1.json

In [4]:
!type src\C1.json

{"contracts":{"src/C1.sol:C1":{"abi":[{"inputs":[],"name":"get","outputs":[{"internalType":"uint128","name":"","type":"uint128"}],"stateMutability":"view","type":"function"},{"inputs":[],"name":"get7","outputs":[{"internalType":"uint128","name":"","type":"uint128"}],"stateMutability":"pure","type":"function"},{"inputs":[{"internalType":"uint128","name":"_v1","type":"uint128"}],"name":"set","outputs":[],"stateMutability":"nonpayable","type":"function"}],"bin":"608060405234801561001057600080fd5b506101e1806100206000396000f3fe608060405234801561001057600080fd5b50600436106100415760003560e01c8063086949b7146100465780634178462f146100645780636d4ce63c14610080575b600080fd5b61004e61009e565b60405161005b9190610158565b60405180910390f35b61007e6004803603810190610079919061011c565b6100a7565b005b6100886100e2565b6040516100959190610158565b60405180910390f35b60006007905090565b806000806101000a8154816fffffffffffffffffffffffffffffffff02191690836fffffffffffffffffffffffffffffffff16021790555050565b600080600090549061

In [5]:
%%writefile src/C1Deploy.js
var Web3 = require('web3');
var web3 = new Web3(new Web3.providers.HttpProvider("http://localhost:8345"));
var fs=require('fs');
var _str = fs.readFileSync("src/C1.json");
var _json = JSON.parse(_str)
//var _abiArray = JSON.parse(_json.contracts.sHello2.abi);
//var _abiArray = JSON.parse(_json.contracts["src/C1.sol:C1"].abi);
var _abiArray = _json.contracts["src/C1.sol:C1"].abi;
//var _bin = _json.contracts.sHello2.bin;
var _bin = "0x"+_json.contracts["src/C1.sol:C1"].bin;

//unlock the account with a password provided
//web3.personal.unlockAccount(web3.eth.accounts[0],'password');
async function deploy() {
    const accounts = await web3.eth.getAccounts();
    console.log("Deploying the contract from " + accounts[0]);
    var deployed = await new web3.eth.Contract(_abiArray)
        .deploy({data: _bin})
        .send({from: accounts[0], gas: 1000000})
        .on('transactionHash', function(hash){
            console.log(">>> transactionHash" + hash);
        })
        .on('receipt', function(receipt){
            console.log(">>> RECEPIT hash: " + receipt.transactionHash + "\n>>> address:" + receipt.contractAddress);
        })
        .on('error', function(error, receipt) {
            console.log(">>> ERROR " + error);
        });
        //.then(function(newContractInstance){
        //    console.log(newContractInstance.options.address)
        //});
    console.log("---> The contract deployed to: " + deployed.options.address)
}
deploy()

Writing src/C1Deploy.js


In [6]:
!node src/C1Deploy.js

Deploying the contract from 0xf88f0fF43C425Fa67B18a31D608d7C6F925b397e
>>> transactionHash0xb05e84a44fb9d6db80fd6bec6ffb0200e7457c48c519ace0d3d10e09c4c7667d
>>> RECEPIT hash: 0xb05e84a44fb9d6db80fd6bec6ffb0200e7457c48c519ace0d3d10e09c4c7667d
>>> address:0x3976EE2383A4cA2Ab2B95618b56C9f1816510591
---> The contract deployed to: 0x3976EE2383A4cA2Ab2B95618b56C9f1816510591


In [7]:
%%writefile src/C1Use.js
var Web3=require('web3');
var web3=new Web3(new Web3.providers.HttpProvider("http://localhost:8345"));
var fs=require('fs');
var _str = fs.readFileSync("src/C1.json");
var _json=JSON.parse(_str)
//var _abiArray=JSON.parse(_json.contracts.sHello2.abi);
//var _abiArray=JSON.parse(_json.contracts["src/C1.sol:C1"].abi);
var _abiArray = _json.contracts["src/C1.sol:C1"].abi;

var c1 = new web3.eth.Contract(_abiArray, "0x3976EE2383A4cA2Ab2B95618b56C9f1816510591");
async function doIt() {
    const accounts = await web3.eth.getAccounts();
    console.log("Account: " + accounts[0]);
    const balanceBefore = await web3.eth.getBalance(accounts[0]);
    console.log("Balance before: " + balanceBefore);
    c1.methods.get7().call().then(console.log);
    await c1.methods.set(9).send({from: accounts[0],gas:50000});
    c1.methods.get().call().then(console.log);
    const balanceAfter = await web3.eth.getBalance(accounts[0]);
    console.log("Balance after: " + balanceAfter);
    console.log("Balance diff: " + (balanceBefore - balanceAfter));
    //hello.methods.kill().send({from: accounts[0]})
}

doIt()

Writing src/C1Use.js


In [8]:
!node src/C1Use.js

Account: 0xf88f0fF43C425Fa67B18a31D608d7C6F925b397e
Balance before: 999999685826000000000
7
Balance after: 999999598260000000000
Balance diff: 87566000062464
9


In [9]:
%%writefile src/C2.sol
//SPDX-License-Identifier: GPL-3.0-or-later
pragma solidity ^0.8.0;
//pragma solidity 0.6;
//pragma solidity 0.4.21;
import "./C1.sol";

contract C2 {
    C1 c1;
    //function C2() public {  //0.4.21 constructor
    constructor() {    //constructor() public {    //0.6 constructor
        c1=new C1();
    }
    function setC1(address _addressOfC1) public {
        c1 = C1(_addressOfC1);
    }
    function set(uint128 _v1) public {
        c1.set(_v1);
    }
    function get() public view returns(uint128) {
        return c1.get();
    }
    function get7() public view returns(uint128) {
        return c1.get7();
    }
    function getC1Address() public view returns(address) {
        return address(c1);
    }
}

Writing src/C2.sol


In [10]:
!solc-windows.exe src/C2.sol --combined-json abi,bin > src/C2.json

In [11]:
!type src\C2.json

{"contracts":{"src/C1.sol:C1":{"abi":[{"inputs":[],"name":"get","outputs":[{"internalType":"uint128","name":"","type":"uint128"}],"stateMutability":"view","type":"function"},{"inputs":[],"name":"get7","outputs":[{"internalType":"uint128","name":"","type":"uint128"}],"stateMutability":"pure","type":"function"},{"inputs":[{"internalType":"uint128","name":"_v1","type":"uint128"}],"name":"set","outputs":[],"stateMutability":"nonpayable","type":"function"}],"bin":"608060405234801561001057600080fd5b506101e1806100206000396000f3fe608060405234801561001057600080fd5b50600436106100415760003560e01c8063086949b7146100465780634178462f146100645780636d4ce63c14610080575b600080fd5b61004e61009e565b60405161005b9190610158565b60405180910390f35b61007e6004803603810190610079919061011c565b6100a7565b005b6100886100e2565b6040516100959190610158565b60405180910390f35b60006007905090565b806000806101000a8154816fffffffffffffffffffffffffffffffff02191690836fffffffffffffffffffffffffffffffff16021790555050565b600080600090549061

In [16]:
%%writefile src/C2Deploy.js
var Web3 = require('web3');
var web3 = new Web3(new Web3.providers.HttpProvider("http://localhost:8345"));
var fs=require('fs');
var _str = fs.readFileSync("src/C2.json");
var _json = JSON.parse(_str)
//var _abiArray = JSON.parse(_json.contracts.sHello2.abi);
//var _abiArray = JSON.parse(_json.contracts["src/C2.sol:C2"].abi);
var _abiArray = _json.contracts["src/C2.sol:C2"].abi;
//var _bin = _json.contracts.sHello2.bin;
var _bin = "0x"+_json.contracts["src/C2.sol:C2"].bin;

//unlock the account with a password provided
//web3.personal.unlockAccount(web3.eth.accounts[0],'password');
async function deploy() {
    const accounts = await web3.eth.getAccounts();
    console.log("Deploying the contract from " + accounts[0]);
    var deployed = await new web3.eth.Contract(_abiArray)
        .deploy({data: _bin})
        .send({from: accounts[0], gas: 1000000})
        .on('transactionHash', function(hash){
            console.log(">>> transactionHash" + hash);
        })
        .on('receipt', function(receipt){
            console.log(">>> RECEPIT hash: " + receipt.transactionHash + "\n>>> address:" + receipt.contractAddress);
        })
        .on('error', function(error, receipt) {
            console.log(">>> ERROR " + error);
        });
        //.then(function(newContractInstance){
        //    console.log(newContractInstance.options.address)
        //});
    console.log("---> The contract deployed to: " + deployed.options.address)
}
deploy()

Overwriting src/C2Deploy.js


In [17]:
!node src/C2Deploy.js

Deploying the contract from 0xf88f0fF43C425Fa67B18a31D608d7C6F925b397e
>>> transactionHash0x42c713d9265fd87995e916dac6fb82d9fcb1e4d5790361aed713606c0fd33d29
>>> RECEPIT hash: 0x42c713d9265fd87995e916dac6fb82d9fcb1e4d5790361aed713606c0fd33d29
>>> address:0xaDa4b830B9cAbD83dA73dEE9736AbaB05487fBd6
---> The contract deployed to: 0xaDa4b830B9cAbD83dA73dEE9736AbaB05487fBd6


In [18]:
%%writefile src/C2Use.js
var Web3=require('web3');
var web3 = new Web3(new Web3.providers.HttpProvider("http://localhost:8345"));
var fs=require('fs');
var _str = fs.readFileSync("src/C2.json");
var _json = JSON.parse(_str)
//var _abiArray = JSON.parse(_json.contracts.sHello2.abi);
//var _abiArray = JSON.parse(_json.contracts["src/C2.sol:C2"].abi);
var _abiArray = _json.contracts["src/C2.sol:C2"].abi;

var c2 = new web3.eth.Contract(_abiArray, "0xaDa4b830B9cAbD83dA73dEE9736AbaB05487fBd6");
async function doIt() {
    const accounts = await web3.eth.getAccounts();
    console.log("Account: " + accounts[0]);
    const balanceBefore = await web3.eth.getBalance(accounts[0]);
    console.log("Balance before: " + balanceBefore);
    console.log("--- new C1 ---");
    await c2.methods.getC1Address().call(function(err, c1addr) {
        if(!err) console.log("c1 address by 'new': "+c1addr);
    });
    c2.methods.get7().call().then(function(res) { console.log("get7(): "+res) });
    console.log("--- set the above deployed address of C1 ---");
    await c2.methods.setC1("0x3976EE2383A4cA2Ab2B95618b56C9f1816510591").send({from:accounts[0], gas:50000});
    await c2.methods.getC1Address().call(function(err, c1addr) {
        if(!err) console.log("c1 address by 'setC1()': "+c1addr);
    });
    c2.methods.get7().call().then(console.log);
    await c2.methods.set(222).send({from: accounts[0],gas:50000});
    c2.methods.get().call().then(console.log);
    const balanceAfter = await web3.eth.getBalance(accounts[0]);
    console.log("Balance after: " + balanceAfter);
    console.log("Balance diff: " + (balanceBefore - balanceAfter));
    //hello.methods.kill().send({from: accounts[0]})
}

doIt()

Overwriting src/C2Use.js


In [19]:
!node src/C2Use.js

Account: 0xf88f0fF43C425Fa67B18a31D608d7C6F925b397e
Balance before: 999997516780000000000
--- new C1 ---
c1 address by 'new': 0xaA880e5CdC9462Aa8261917D2aF6Ef5462a2812F
--- set the above deployed address of C1 ---
get7(): 7
c1 address by 'setC1()': 0x3976EE2383A4cA2Ab2B95618b56C9f1816510591
7
Balance after: 999997403928000000000
Balance diff: 112852000047104
222


In [20]:
!node src/C2Use.js

Account: 0xf88f0fF43C425Fa67B18a31D608d7C6F925b397e
Balance before: 999997403928000000000
--- new C1 ---
c1 address by 'new': 0x3976EE2383A4cA2Ab2B95618b56C9f1816510591
--- set the above deployed address of C1 ---
get7(): 7
c1 address by 'setC1()': 0x3976EE2383A4cA2Ab2B95618b56C9f1816510591
7
Balance after: 999997296676000000000
Balance diff: 107251999965184
222


## SquareArea

In [27]:
%%writefile src/SquareArea.sol
//SPDX-License-Identifier: GPL-3.0-or-later
pragma solidity ^0.8.0;
//pragma solidity ^0.6.0;
//pragma solidity 0.4.21;

contract Square {
    uint128 length;
    function getLength() public view returns(uint128) {
        return length;
    }
    function setLength(uint128 _length) public {
        length=_length;
    }
    function getDegree() public pure returns(uint128) {
        return 90;
    }
}

contract Area {
    Square s;
    address owner;
    //event PrintLog(uint128);
    //function Area() public {
    constructor() { //constructor() public {
        s = new Square();
        owner = msg.sender;
    }
    function changeSquare(address _addressOfSquare) public {
        s=Square(_addressOfSquare);
    }
    function calcArea() view public returns(uint128) {
        uint128 length = s.getLength();
        uint128 area = length*length;
        //emit PrintLog(area);
        return area;
    }
    function setLength(uint128 _length) public {
        s.setLength(_length);
    }
    function getLength() public view returns(uint128) {
        return s.getLength();
    }
    function getDegree() public view returns(uint128) {
        return s.getDegree();
    }
    function getAddressOfSquare() public view returns(address) {
        return address(s);
    }
}

Writing src/SquareArea.sol


In [28]:
!solc-windows.exe src/SquareArea.sol --combined-json abi,bin > src/SquareArea.json

In [29]:
!type src\SquareArea.json

{"contracts":{"src/SquareArea.sol:Area":{"abi":[{"inputs":[],"stateMutability":"nonpayable","type":"constructor"},{"inputs":[],"name":"calcArea","outputs":[{"internalType":"uint128","name":"","type":"uint128"}],"stateMutability":"view","type":"function"},{"inputs":[{"internalType":"address","name":"_addressOfSquare","type":"address"}],"name":"changeSquare","outputs":[],"stateMutability":"nonpayable","type":"function"},{"inputs":[],"name":"getAddressOfSquare","outputs":[{"internalType":"address","name":"","type":"address"}],"stateMutability":"view","type":"function"},{"inputs":[],"name":"getDegree","outputs":[{"internalType":"uint128","name":"","type":"uint128"}],"stateMutability":"view","type":"function"},{"inputs":[],"name":"getLength","outputs":[{"internalType":"uint128","name":"","type":"uint128"}],"stateMutability":"view","type":"function"},{"inputs":[{"internalType":"uint128","name":"_length","type":"uint128"}],"name":"setLength","outputs":[],"stateMutability":"nonpayable","type":

In [30]:
%%writefile src/SquareAreaDeploy.js
var Web3 = require('web3');
var web3 = new Web3(new Web3.providers.HttpProvider("http://localhost:8345"));
var fs=require('fs');
var _str = fs.readFileSync("src/SquareArea.json");
var _json=JSON.parse(_str)
//var _abiArray = JSON.parse(_json.contracts.sHello2.abi);
//var _abiArray = JSON.parse(_json.contracts["src/SquareArea.sol:Area"].abi);
var _abiArray = _json.contracts["src/SquareArea.sol:Area"].abi;
//var _bin = _json.contracts.sHello2.bin;
var _bin = "0x" + _json.contracts["src/SquareArea.sol:Area"].bin;

//unlock the account with a password provided
//web3.personal.unlockAccount(web3.eth.accounts[0],'password');
async function deploy() {
    const accounts = await web3.eth.getAccounts();
    console.log("Deploying the contract from " + accounts[0]);
    var deployed = await new web3.eth.Contract(_abiArray)
        .deploy({data: _bin})
        .send({from: accounts[0], gas: 1000000})
        .on('transactionHash', function(hash){
            console.log(">>> transactionHash"+hash);
        })
        .on('receipt', function(receipt){
            console.log(">>> RECEPIT hash: " + receipt.transactionHash + "\n>>> address:" + receipt.contractAddress);
        })
        .on('error', function(error, receipt) {
            console.log(">>> ERROR "+error);
        });
        //.then(function(newContractInstance){
        //    console.log(newContractInstance.options.address)
        //});
    console.log("---> The contract deployed to: " + deployed.options.address)
}

deploy()

Writing src/SquareAreaDeploy.js


In [31]:
!node src/SquareAreaDeploy.js

Deploying the contract from 0xf88f0fF43C425Fa67B18a31D608d7C6F925b397e
>>> transactionHash0x89e62387068f712574c3318ae73ed3f322250ab3f5f4e8b060fd4861b2905382
>>> RECEPIT hash: 0x89e62387068f712574c3318ae73ed3f322250ab3f5f4e8b060fd4861b2905382
>>> address:0x0dE2F94898064091Ae3e9a285A095EB4F6ec3D4B
---> The contract deployed to: 0x0dE2F94898064091Ae3e9a285A095EB4F6ec3D4B


In [42]:
%%writefile src/SquareAreaUse.js
var Web3 = require('web3');
//var web3 = new Web3(new Web3.providers.HttpProvider("http://117.16.44.45:8345"));         //nok
//var web3 = new Web3(new Web3.providers.WebsocketProvider("http://117.16.44.45:8345"));  //ok
var web3 = new Web3(new Web3.providers.WebsocketProvider("ws://localhost:8345"));
var fs=require('fs');
var _str = fs.readFileSync("src/SquareArea.json");
var _json = JSON.parse(_str)
//var _abiArray = JSON.parse(_json.contracts.sHello2.abi);
//var _abiArray = JSON.parse(_json.contracts["src/SquareArea.sol:Area"].abi);
var _abiArray = _json.contracts["src/SquareArea.sol:Area"].abi;

async function doIt() {
    var area = new web3.eth.Contract(_abiArray, "0x0dE2F94898064091Ae3e9a285A095EB4F6ec3D4B");
    var speed;
    const accounts = await web3.eth.getAccounts();
    console.log("Account: " + accounts[0]);
    const balanceBefore = await web3.eth.getBalance(accounts[0]);
    console.log("Balance before: " + balanceBefore);
    area.methods.getDegree().call().then(console.log);
    await area.methods.setLength(9).send({from: accounts[0]});
    area.methods.getLength().call().then(console.log);
    //area.methods.calcArea().send({from: accounts[0]});
    area.methods.calcArea().call().then(console.log);
    //area.methods.getAddressOfSquare().call().then(function(address) {
    //    console.log("Square Address: " + address);
    //});
    const addr = await area.methods.getAddressOfSquare().call();
    console.log("Square Address: " + addr);
    const balanceAfter = await web3.eth.getBalance(accounts[0]);
    console.log("Balance after: " + balanceAfter);
    console.log("Balance diff: " + (balanceBefore - balanceAfter));
    process.exit(1); //force exit -> may terminate some functions (speedDownBy10, getSpeed)  
}

doIt()

Overwriting src/SquareAreaUse.js


In [43]:
!node src/SquareAreaUse.js

Account: 0xf88f0fF43C425Fa67B18a31D608d7C6F925b397e
Balance before: 999994623128000000000
90
Square Address: 0x7A262daF562F4041f32027b12CdcA362F0465A8f
9
81
Balance after: 999994564228000000000
Balance diff: 58900000014336


In [44]:
%%writefile src/Square.sol
//SPDX-License-Identifier: GPL-3.0-or-later
pragma solidity ^0.8.0;
//pragma solidity ^0.6.0;
//pragma solidity 0.4.21;

contract Square {
    uint128 length;
    function getLength() public view returns(uint128) {
        return length;
    }
    function setLength(uint128 _length) public {
        length=_length;
    }
    function getDegree() public pure returns(uint128) {
        return 90;
    }
}

Overwriting src/Square.sol


In [45]:
%%writefile src/Area.sol
//SPDX-License-Identifier: GPL-3.0-or-later
pragma solidity ^0.8.0;
//pragma solidity ^0.6.0;
//pragma solidity 0.4.21;

import "./Square.sol";

contract Area {
    Square s;
    address owner;
    //event PrintLog(uint128);
    //function Area() public {  //0.4.21
    constructor() { //constructor() public {
        s = new Square();
        owner = msg.sender;
    }
    function changeSquare(address _addressOfSquare) public {
        s=Square(_addressOfSquare);
    }
    function calcArea() view public returns(uint128) {
        uint128 length = s.getLength();
        uint128 area = length*length;
        //emit PrintLog(area);
        return area;
    }
    function setLength(uint128 _length) public {
        s.setLength(_length);
    }
    function getLength() public view returns(uint128) {
        return s.getLength();
    }
    function getDegree() public view returns(uint128) {
        return s.getDegree();
    }
    function getAddressOfSquare() public view returns(address) {
        return address(s);
    }
}

Overwriting src/Area.sol


In [46]:
!solc-windows.exe src/Area.sol --combined-json abi,bin > src/Area.json

In [47]:
!type src\Area.json

{"contracts":{"src/Area.sol:Area":{"abi":[{"inputs":[],"stateMutability":"nonpayable","type":"constructor"},{"inputs":[],"name":"calcArea","outputs":[{"internalType":"uint128","name":"","type":"uint128"}],"stateMutability":"view","type":"function"},{"inputs":[{"internalType":"address","name":"_addressOfSquare","type":"address"}],"name":"changeSquare","outputs":[],"stateMutability":"nonpayable","type":"function"},{"inputs":[],"name":"getAddressOfSquare","outputs":[{"internalType":"address","name":"","type":"address"}],"stateMutability":"view","type":"function"},{"inputs":[],"name":"getDegree","outputs":[{"internalType":"uint128","name":"","type":"uint128"}],"stateMutability":"view","type":"function"},{"inputs":[],"name":"getLength","outputs":[{"internalType":"uint128","name":"","type":"uint128"}],"stateMutability":"view","type":"function"},{"inputs":[{"internalType":"uint128","name":"_length","type":"uint128"}],"name":"setLength","outputs":[],"stateMutability":"nonpayable","type":"funct

In [48]:
%%writefile src/AreaDeploy.js
var Web3 = require('web3');
var web3 = new Web3(new Web3.providers.HttpProvider("http://localhost:8345"));
var fs = require('fs');
var _str = fs.readFileSync("src/Area.json");
var _json = JSON.parse(_str)
//var _abiArray = JSON.parse(_json.contracts.sHello2.abi);
//var _abiArray = JSON.parse(_json.contracts["src/Area.sol:Area"].abi);
var _abiArray = _json.contracts["src/Area.sol:Area"].abi;
//var _bin = _json.contracts.sHello2.bin;
var _bin = "0x" + _json.contracts["src/Area.sol:Area"].bin;

//unlock the account with a password provided
//web3.personal.unlockAccount(web3.eth.accounts[0],'password');
async function deploy() {
    const accounts = await web3.eth.getAccounts();
    console.log("Deploying the contract from " + accounts[0]);
    var deployed = await new web3.eth.Contract(_abiArray)
        .deploy({data: _bin})
        .send({from: accounts[0], gas: 1000000}, function(err, transactionHash) {
            if(!err) console.log("hash: " + transactionHash); 
        })
        //.then(function(newContractInstance){
        //    console.log(newContractInstance.options.address)
        //});
    console.log("---> The contract deployed to: " + deployed.options.address)
}

deploy()

Overwriting src/AreaDeploy.js


In [49]:
!node src/AreaDeploy.js

Deploying the contract from 0xf88f0fF43C425Fa67B18a31D608d7C6F925b397e
hash: 0x919f4f6482aee5a276166e66658373db3af05beef34a7f0a5a66951d22864747
---> The contract deployed to: 0xaCf37c2Ff04c20f62DC48c4a67B0B15F46390468


In [50]:
%%writefile src/AreaUse.js
var Web3=require('web3');
var web3=new Web3(new Web3.providers.HttpProvider("http://localhost:8345"));       //nok
//var web3 = new Web3(new Web3.providers.WebsocketProvider("http://117.16.44.45:8345"));  //ok
//var web3 = new Web3(new Web3.providers.WebsocketProvider("ws://117.16.44.45:8345"));  //ok
var fs=require('fs');
var _str = fs.readFileSync("src/Area.json");
var _json = JSON.parse(_str)
//var _abiArray = JSON.parse(_json.contracts.Area.abi);
//var _abiArray = JSON.parse(_json.contracts["src/Area.sol:Area"].abi);
var _abiArray = _json.contracts["src/Area.sol:Area"].abi;

async function doIt() {
    var hello = new web3.eth.Contract(_abiArray, "0xaCf37c2Ff04c20f62DC48c4a67B0B15F46390468");
    const accounts = await web3.eth.getAccounts();
    console.log("Account: " + accounts[0]);
    const balanceBefore = await web3.eth.getBalance(accounts[0]);
    console.log("Balance before: " + balanceBefore);
    // do by the Square address as set in the constructor
    hello.methods.getAddressOfSquare().call(function(err, c1addr) {
        if(!err) console.log(">> Square address by 'new': "+c1addr);
    });
    await hello.methods.setLength(10).send({from: accounts[0]});
    hello.methods.getLength().call().then(console.log);
    hello.methods.calcArea().call().then(console.log);
    hello.methods.getDegree().call().then(console.log);
    hello.methods.getAddressOfSquare().call().then(console.log);
    //redo by the Square address as changed by changeSquare()
    await hello.methods.changeSquare('0x7A262daF562F4041f32027b12CdcA362F0465A8f').send({from: accounts[0]});
    hello.methods.getAddressOfSquare().call(function(err, c1addr) {
        if(!err) console.log(">> Square address by 'changeSquare: "+c1addr);
    });
    await hello.methods.setLength(10).send({from: accounts[0]});
    hello.methods.getLength().call().then(console.log);
    hello.methods.calcArea().call().then(console.log);
    hello.methods.getDegree().call().then(console.log);
    hello.methods.getAddressOfSquare().call().then(console.log);
    const balanceAfter = await web3.eth.getBalance(accounts[0]);
    console.log("Balance after: " + balanceAfter);
    console.log("Balance diff: " + (balanceBefore - balanceAfter));
}

doIt()

Writing src/AreaUse.js


In [51]:
!node src/AreaUse.js

Account: 0xf88f0fF43C425Fa67B18a31D608d7C6F925b397e
Balance before: 999993386358000000000
>> Square address by 'new': 0xCACeD09B5D5c4F265eBDA0b04c40b8a60D043BDd
0xCACeD09B5D5c4F265eBDA0b04c40b8a60D043BDd
90
10
100
>> Square address by 'changeSquare: 0x7A262daF562F4041f32027b12CdcA362F0465A8f
Balance after: 999993169074000000000
Balance diff: 217284000088064
0x7A262daF562F4041f32027b12CdcA362F0465A8f
90
10
100
